* [x] Get data into csv to work in pandas
* [x] Perform sentiment classification with Hugging Face model (2024 SOTA)
* [ ] Extract from text key words that resulted in sentiment class (entity-based sentiment analysis)
* [ ] topic modeling?
* [ ] other advanced analytics for reviews using LLM/HuggingFace
* [ ] compare competition side-by-side (Japanese restaurants)
* [ ] get criteria for ranking/qualities in restaurant based on aggregation of common words through clustering

In [1]:
!pip install duckdb jupysql -q transformers --quiet 

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import duckdb

%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

import re
from transformers import pipeline, AutoTokenizer

from nltk.sentiment.vader import SentimentIntensityAnalyzer

Deploy Shiny apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


2024-05-28 01:55:31.991746: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 01:55:31.991875: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 01:55:32.157043: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
dfr = pd.read_csv('/kaggle/input/yelp-revised/yelp_biz_rev.csv')
dfr.head()

/tmp/ipykernel_18/3121522079.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  dfr = pd.read_csv('/kaggle/input/yelp-revised/yelp_biz_rev.csv')


,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,review_id,user_id,review_stars,text,date
0,0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80.0,"{'key': ['RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking', 'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters'], 'value': ['False', 'False', 'False', ""{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}"", 'True', '1', 'True', 'False', ""u'free'"", ""u'none'"", 'True']}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries",BXQcBN0iAi1lAUxibGLFzA,6_SpY41LIHZuIaiDs5FMKA,4.0,"This is nice little Chinese bakery in the heart of Philadelphia's Chinatown! The female cashier was very friendly (flirtatious!) and the pastries shown in nicely adorned display cases. I stopped by early one evening had a sesame ball, which was filled with bean paste. The glutinous rice of the ball was nicely flavored, similar to Bai Tang Gao. Definitely as place worth stopping at if you are in the area.",2014-05-26 01:09:53
1,1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80.0,"{'key': ['RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking', 'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters'], 'value': ['False', 'False', 'False', ""{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}"", 'True', '1', 'True', 'False', ""u'free'"", ""u'none'"", 'True']}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries",uduvUCvi9w3T2bSGivCfXg,tCXElwhzekJEH6QJe3xs7Q,4.0,"This is the bakery I usually go to in Chinatown. They have a decent variety of buns and pastries. Red bean-filled buns, mung bean-filled buns, coconut stuff, egg tarts etc. If you buy a dozen, they will give you two more for free. The prices are standard for a Chinese bakery, with most things at approximately $1. The service has always been good. They do serve coffee and tea. There are a couple of tables inside, with usually the expected group of older Chinese guys chillin' in the corner table debating about something. Great place to stock up on the buns, which I usually do before leaving town.",2013-10-05 15:19:06
2,2,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80.0,"{'key': ['RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking', 'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters'], 'value': ['False', 'False', 'False', ""{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}"", 'True', '1', 'True', 'False', ""u'free'"", ""u'none'"", 'True']}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries",a0vwPOqDXXZuJkbBW2356g,WqfKtI-aGMmvbA9pPUxNQQ,5.0,"A delightful find in Chinatown! Very clean, and very kind service. Everyone always seems happy in here, and their treats are absolutely delicious. You can also get a ton of sweet treats without any judgement! They're my favorite.",2013-10-25 01:34:57
3,3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80.0,"{'key': ['RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking', 'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters'], 'value': ['False', 'False', 'False', ""{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}"", 'True', '1', 'True', 'False', ""u'free'"", ""u'none'"", 'True']}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries",MKNp_CdR2k2202-c8GN5Dw,3-1va0IQfK-9tUMzfHWfTA,5.0,I ordered a graduation cake for my niece and 

In [4]:
query = """
select review_id
, review_stars
, text
from dfr
"""

df_sent2 = duckdb.query(query).df()
df_sent2

,review_id,review_stars,text
0,BXQcBN0iAi1lAUxibGLFzA,4.0,"This is nice little Chinese bakery in the heart of Philadelphia's Chinatown! The female cashier was very friendly (flirtatious!) and the pastries shown in nicely adorned display cases. I stopped by early one evening had a sesame ball, which was filled with bean paste. The glutinous rice of the ball was nicely flavored, similar to Bai Tang Gao. Definitely as place worth stopping at if you are in the area."
1,uduvUCvi9w3T2bSGivCfXg,4.0,"This is the bakery I usually go to in Chinatown. They have a decent variety of buns and pastries. Red bean-filled buns, mung bean-filled buns, coconut stuff, egg tarts etc. If you buy a dozen, they will give you two more for free. The prices are standard for a Chinese bakery, with most things at approximately $1. The service has always been good. They do serve coffee and tea. There are a couple of tables inside, with usually the expected group of older Chinese guys chillin' in the corner table debating about something. Great place to stock up on the buns, which I usually do before leaving town."
2,a0vwPOqDXXZuJkbBW2356g,5.0,"A delightful find in Chinatown! Very clean, and very kind service. Everyone always seems happy in here, and their treats are absolutely delicious. You can also get a ton of sweet treats without any judgement! They're my favorite."
3,MKNp_CdR2k2202-c8GN5Dw,5.0,I ordered a graduation cake for my niece and it came out absolutely perfect. Will come again for future events! Staff was nice also and the cake was really good.
4,D1GisLDPe84Rrk_R4X2brQ,4.0,"HK-STYLE MILK TEA: FOUR STARS\n\nNot quite sure why there are two Saint Honore Pastries business listings on Yelp, but I have been checking in to this one.\n\nBoyfriend and I go by it all the time while in Chinatown, but it never crossed our mind to try it. I think we might have tried it once a year or two ago, but it wasn't memorable. I decided to try the HK-style milk tea not too long ago and was hooked. I think I went here almost every day one week to get the milk tea, only to realize that I should REALLY watch my sugar intake...it's sooo good! \n\nI personally don't think it's better than Zhong Gang's HK-style milk tea because they have their off days when a certain lady makes them (not enough ice, too much ice, etc.).\n\nOn a side note: I wanted to try the crispy roast pork bun, but accidentally said the wrong name in Chinese and got the roast pork bun. It was decent enough, although I will get to trying the crispy roast pork bun soon! \n\nI will be back~"
...,...,...,...
3773767,Kt3gFeW1rhZz7RuiV-6Tcw,5.0,"This is my favorite food truck! I only wish I lived closer to Philly so that I could eat here more often. The prices are affordable, the portions are generous, the staff is friendly, and the food is AMAZING. \n\nThe last time I went to the Plum Pit, I ordered a jerk chicken sandwich to split with a friend. Learn from our mistake--order your own sandwich, and if you can't finish it, save half for later. We ended up going back for another sandwich (the one with chicken tenders, mac & cheese, and bacon--I can't remember the name) anyway. We also got two orders of bread pudding to bring home, which, though pretty good cold, is ever better when you heat it up in the oven for 10-15 minutes."
3773768,ruy3Ycey_gGbwkE_3TX1Fg,5.0,"This food truck was stupid. Stupidly delicious. \n\nShould be a 4.5 because they were a little slow compared to other trucks, but once the food showed up, totally ON POINT.\n\nWe had the bacon Mac and cheese with the French fry bowl, topped with jerk chicken. I could not get enough of the jerk chicken. And the bacon Mac and cheese was super creamy ( again 4.5 we saw a big can of nacho cheese instead of good cheese "" . \n\nWould definitely eat here again if they were at the Stables."
3773769,C_l8NTpvNOEUorEmEOusaA,5.0,Bubba never disappoints i go to his fb page and hunt him down and now i have my friends and family stalking him too. One of the main r

In [5]:
# reduce file size by getting sample
#sample 500,000 to use less RAM
sample = df_sent2.sample(n=500000)
len(sample)

500000

In [6]:
sample.head(1)

,review_id,review_stars,text
2123287,Gjpz-PtFah1Lf-aH9R9lYQ,3.0,The doubles were a bit soggy and just ok on flavor but the fried shark bake sandwich was really good. Over all nothing really blew my socks off but I'd come back for the shark.


# Text Pre-Processing

In [7]:
# text column needs to be changed to string...
sample['text'] = sample['text'].apply(str)

In [8]:
# Get code to clean text of HTML tags, punctuation
def cleanhtml(x): 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', x) 
    return cleantext.replace('\n\n', ' ').replace('\n', '')
    
sample['clean_text'] = sample.apply(lambda row: cleanhtml(row['text']), axis=1)

# Examine sample of dirty comments to see if translation went through
sample

,review_id,review_stars,text,clean_text
2123287,Gjpz-PtFah1Lf-aH9R9lYQ,3.0,The doubles were a bit soggy and just ok on flavor but the fried shark bake sandwich was really good. Over all nothing really blew my socks off but I'd come back for the shark.,The doubles were a bit soggy and just ok on flavor but the fried shark bake sandwich was really good. Over all nothing really blew my socks off but I'd come back for the shark.
1248547,dXGbvuw7yB8j4iA3K6Wq-g,5.0,"Awesome food, awesome service, ready to go back - appetizers were on point, we had sausage and cheese plate - sandwiches were great, chicken enchilada and roast beef were better than the pizza sando, drinks even better - loved it!","Awesome food, awesome service, ready to go back - appetizers were on point, we had sausage and cheese plate - sandwiches were great, chicken enchilada and roast beef were better than the pizza sando, drinks even better - loved it!"
2919259,W5S6tI-QkN1JE9CKRIn_yQ,5.0,"Great service, friendly energetic staff, huge portions, outstanding food, great selection, free wi-fi and a fun funky environment! Highly recommended. Always my go to place either before or after donating blood next door. Pretty sure their claim of best omelettes on the planet is totally justified!","Great service, friendly energetic staff, huge portions, outstanding food, great selection, free wi-fi and a fun funky environment! Highly recommended. Always my go to place either before or after donating blood next door. Pretty sure their claim of best omelettes on the planet is totally justified!"
2699090,rKnQRoYXforTz7eX8-eEgA,5.0,"Absolutely, positively an amazing experience! The atmosphere and food was nothing short of first class! Our waitress Mandy was superb and her expertise in food choices and drinks was second to none! The decadent appetizers she recommended was mouth watering and left us in awe of what was to come. We decided to take her vast knowledge of the menu and ordered the pawn in salmon pasta and the pork dish Gucci. Needless to say every single morsel was consumed with a smile on our faces! Mandy was the cherry on the top of our dining experience at twigs! Give this woman a raise now! See my photos... I guarantee you will not be disappointed!","Absolutely, positively an amazing experience! The atmosphere and food was nothing short of first class! Our waitress Mandy was superb and her expertise in food choices and drinks was second to none! The decadent appetizers she recommended was mouth watering and left us in awe of what was to come. We decided to take her vast knowledge of the menu and ordered the pawn in salmon pasta and the pork dish Gucci. Needless to say every single morsel was consumed with a smile on our faces! Mandy was the cherry on the top of our dining experience at twigs! Give this woman a raise now! See my photos... I guarantee you will not be disappointed!"
3723072,Ri0TDGEkovaaiFxBs3iKQA,2.0,"I was looking forward to trying burgerfi. I won't be back. Service was great but the burger had no flavor, the bottom of the bun was hard. Fries were ok and the shake was lovely.","I was looking forward to trying burgerfi. I won't be back. Service was great but the burger had no flavor, the bottom of the bun was hard. Fries were ok and the shake was lovely."
...,...,...,...,...
2424094,wNSyeZx_sr8C9DTOuoVnbg,5.0,"Seriously delicious! The restaurant is filled with the aroma of tender smoked bacon....mmm be still my beating heart.\n\nThe pork cheeks were so tasty! The ribs were tender and juicy! The jalapeno sweetbread makes me want to hunt down the chef and attack him! Ok...he totally deserves that James Beard award!\n\nThe decor is modern with touch of trendy eco-cool. Green is the new black, right?\n\nSeriously fine cuisine that'll knock the jambalaya off your socks!","Seriously delicious! The restaurant is filled with the aroma of tender smoked bacon....mmm be still my beating heart. The pork cheeks were so tasty! The ribs were tender and juicy! The jalape

In [9]:
# normalize text by lowering it all
sample['clean_text'] = sample['clean_text'].str.lower()

In [10]:
%%time
# Due to transformer error:
# If you are running a transformer model with HuggingFace, there is a chance that one of the input sentences is longer than 512 tokens. Either truncate or split your sentences. I suspect the shorter sentences are padded to 512 tokens.

#1. get token length/total words with max: 500 total words as max length since tokenizer can add suffixes as a token
#2. eliminate tokens lengths greater than max/summarize text using chatgpt?

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# sentence = "Sentence to check whether it exceeds 512 tokens"

# tokenized_sentence = tokenizer.tokenize(sentence)

# print(len(sentence.split()))  # here is the default length of the sentence
# print(len(tokenized_sentence)) # here is the tokenized length

## Example applied to df

sample['num_tokens_tok'] = sample.apply(lambda row: len(tokenizer.tokenize(row.clean_text)), axis = 1)
sample.head()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (596 > 512). Running this sequence through the model will result in indexing errors


CPU times: user 4min 59s, sys: 76.7 ms, total: 4min 59s
Wall time: 5min 2s


,review_id,review_stars,text,clean_text,num_tokens_tok
2123287,Gjpz-PtFah1Lf-aH9R9lYQ,3.0,The doubles were a bit soggy and just ok on flavor but the fried shark bake sandwich was really good. Over all nothing really blew my socks off but I'd come back for the shark.,the doubles were a bit soggy and just ok on flavor but the fried shark bake sandwich was really good. over all nothing really blew my socks off but i'd come back for the shark.,41
1248547,dXGbvuw7yB8j4iA3K6Wq-g,5.0,"Awesome food, awesome service, ready to go back - appetizers were on point, we had sausage and cheese plate - sandwiches were great, chicken enchilada and roast beef were better than the pizza sando, drinks even better - loved it!","awesome food, awesome service, ready to go back - appetizers were on point, we had sausage and cheese plate - sandwiches were great, chicken enchilada and roast beef were better than the pizza sando, drinks even better - loved it!",52
2919259,W5S6tI-QkN1JE9CKRIn_yQ,5.0,"Great service, friendly energetic staff, huge portions, outstanding food, great selection, free wi-fi and a fun funky environment! Highly recommended. Always my go to place either before or after donating blood next door. Pretty sure their claim of best omelettes on the planet is totally justified!","great service, friendly energetic staff, huge portions, outstanding food, great selection, free wi-fi and a fun funky environment! highly recommended. always my go to place either before or after donating blood next door. pretty sure their claim of best omelettes on the planet is totally justified!",60
2699090,rKnQRoYXforTz7eX8-eEgA,5.0,"Absolutely, positively an amazing experience! The atmosphere and food was nothing short of first class! Our waitress Mandy was superb and her expertise in food choices and drinks was second to none! The decadent appetizers she recommended was mouth watering and left us in awe of what was to come. We decided to take her vast knowledge of the menu and ordered the pawn in salmon pasta and the pork dish Gucci. Needless to say every single morsel was consumed with a smile on our faces! Mandy was the cherry on the top of our dining experience at twigs! Give this woman a raise now! See my photos... I guarantee you will not be disappointed!","absolutely, positively an amazing experience! the atmosphere and food was nothing short of first class! our waitress mandy was superb and her expertise in food choices and drinks was second to none! the decadent appetizers she recommended was mouth watering and left us in awe of what was to come. we decided to take her vast knowledge of the menu and ordered the pawn in salmon pasta and the pork dish gucci. needless to say every single morsel was consumed with a smile on our faces! mandy was the cherry on the top of our dining experience at twigs! give this woman a raise now! see my photos... i guarantee you will not be disappointed!",134
3723072,Ri0TDGEkovaaiFxBs3iKQA,2.0,"I was looking forward to trying burgerfi. I won't be back. Service was great but the burger had no flavor, the bottom of the bun was hard. Fries were ok and the shake was lovely.","i was looking forward to trying burgerfi. i won't be back. service was great but the burger had no flavor, the bottom of the bun was hard. fries were ok and the shake was lovely.",43


In [11]:
# get rid of rows that are <450 words in length since this tokenizer is approx of tokenizer used for sentiment model
query = """
select *
from sample
where num_tokens_tok < 450
"""

sample_short_rev = duckdb.query(query).df()
sample_short_rev.head()

,review_id,review_stars,text,clean_text,num_tokens_tok
0,Gjpz-PtFah1Lf-aH9R9lYQ,3.0,The doubles were a bit soggy and just ok on flavor but the fried shark bake sandwich was really good. Over all nothing really blew my socks off but I'd come back for the shark.,the doubles were a bit soggy and just ok on flavor but the fried shark bake sandwich was really good. over all nothing really blew my socks off but i'd come back for the shark.,41
1,dXGbvuw7yB8j4iA3K6Wq-g,5.0,"Awesome food, awesome service, ready to go back - appetizers were on point, we had sausage and cheese plate - sandwiches were great, chicken enchilada and roast beef were better than the pizza sando, drinks even better - loved it!","awesome food, awesome service, ready to go back - appetizers were on point, we had sausage and cheese plate - sandwiches were great, chicken enchilada and roast beef were better than the pizza sando, drinks even better - loved it!",52
2,W5S6tI-QkN1JE9CKRIn_yQ,5.0,"Great service, friendly energetic staff, huge portions, outstanding food, great selection, free wi-fi and a fun funky environment! Highly recommended. Always my go to place either before or after donating blood next door. Pretty sure their claim of best omelettes on the planet is totally justified!","great service, friendly energetic staff, huge portions, outstanding food, great selection, free wi-fi and a fun funky environment! highly recommended. always my go to place either before or after donating blood next door. pretty sure their claim of best omelettes on the planet is totally justified!",60
3,rKnQRoYXforTz7eX8-eEgA,5.0,"Absolutely, positively an amazing experience! The atmosphere and food was nothing short of first class! Our waitress Mandy was superb and her expertise in food choices and drinks was second to none! The decadent appetizers she recommended was mouth watering and left us in awe of what was to come. We decided to take her vast knowledge of the menu and ordered the pawn in salmon pasta and the pork dish Gucci. Needless to say every single morsel was consumed with a smile on our faces! Mandy was the cherry on the top of our dining experience at twigs! Give this woman a raise now! See my photos... I guarantee you will not be disappointed!","absolutely, positively an amazing experience! the atmosphere and food was nothing short of first class! our waitress mandy was superb and her expertise in food choices and drinks was second to none! the decadent appetizers she recommended was mouth watering and left us in awe of what was to come. we decided to take her vast knowledge of the menu and ordered the pawn in salmon pasta and the pork dish gucci. needless to say every single morsel was consumed with a smile on our faces! mandy was the cherry on the top of our dining experience at twigs! give this woman a raise now! see my photos... i guarantee you will not be disappointed!",134
4,Ri0TDGEkovaaiFxBs3iKQA,2.0,"I was looking forward to trying burgerfi. I won't be back. Service was great but the burger had no flavor, the bottom of the bun was hard. Fries were ok and the shake was lovely.","i was looking forward to trying burgerfi. i won't be back. service was great but the burger had no flavor, the bottom of the bun was hard. fries were ok and the shake was lovely.",43


In [12]:
#Check max number tokens
query = """
select max(num_tokens_tok) as max_tokens
from sample_short_rev
"""

duckdb.query(query).df()

,max_tokens
0,449


In [13]:
# remaining of sample after token length filter
len(sample_short_rev)

488953

# Sentiment Classification

In [14]:
# using Hugging Face sentiment classifier trained on Yelp restaurant reviews

# Set up the inference pipeline using a model from the 🤗 Hub

sentiment_analysis = pipeline(model="mrcaelumn/yelp_restaurant_review_sentiment_analysis")

config.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [15]:
test = sample_short_rev.sample(n=5000)

In [16]:
%%time

test = (
test.assign(sentiment = lambda x: x['clean_text'].apply(lambda s: sentiment_analysis(s)))
    .assign(
         label = lambda x: x['sentiment'].apply(lambda s: (s[0]['label'])),
         score = lambda x: x['sentiment'].apply(lambda s: (s[0]['score']))
    )
)

test.head()

CPU times: user 17min 37s, sys: 2.08 s, total: 17min 39s
Wall time: 8min 50s


,review_id,review_stars,text,clean_text,num_tokens_tok,sentiment,label,score
208634,bZxahkC542f7D6xlDSLsnA,5.0,Our waiter Winston was kind and attentive to our visit. WOW for Winston!! Five stars.,our waiter winston was kind and attentive to our visit. wow for winston!! five stars.,21,"[{'label': 'LABEL_2', 'score': 0.9975969195365906}]",LABEL_2,0.997597
392519,p1-uV5Iv97kUI5zqIJSy5A,5.0,"Who hasn't hung out at Dargan's, I mean really! This is a great place to get in touch with your Irish roots, though for me those are distant roots but just a hop and a skip on a ferry from England where I couldn't get a better serving of corn beef and cabbage if I searched out the finest of pups and all the land, Dargan's is Irie my friends. On St. Patrick's Day, when in Santa Barbara, it is a tradition to get you some shots of Jamison's and green beer at the James Joyce O'Malley's and/or Dargan's!!","who hasn't hung out at dargan's, i mean really! this is a great place to get in touch with your irish roots, though for me those are distant roots but just a hop and a skip on a ferry from england where i couldn't get a better serving of corn beef and cabbage if i searched out the finest of pups and all the land, dargan's is irie my friends. on st. patrick's day, when in santa barbara, it is a tradition to get you some shots of jamison's and green beer at the james joyce o'malley's and/or dargan's!!",135,"[{'label': 'LABEL_2', 'score': 0.9966173768043518}]",LABEL_2,0.996617
346834,tdGwUnyIg6Y1gXKF1iMRkg,2.0,"This was my second time at Sabrina's during the Covid crisis. Both times I waited, but this time I was in line for over 2 1/2 hours. When I was finally seated, it took the waitress very long for me and my guest to even get water. I ordered the Santa Fe turkey burger. It was so extremely salty, that I was not able to eat it. I joked with my friend about not only the saltiness of the meal, but of the 3 pieces of black beans that were placed on the patty. When I asked to have it replaced, the turkey burger came with no beans, and a few pieces of corn. This was an overpriced turkey burger that was definitely NOT worth the wait.","this was my second time at sabrina's during the covid crisis. both times i waited, but this time i was in line for over 2 1/2 hours. when i was finally seated, it took the waitress very long for me and my guest to even get water. i ordered the santa fe turkey burger. it was so extremely salty, that i was not able to eat it. i joked with my friend about not only the saltiness of the meal, but of the 3 pieces of black beans that were placed on the patty. when i asked to have it replaced, the turkey burger came with no beans, and a few pieces of corn. this was an overpriced turkey burger that was definitely not worth the wait.",150,"[{'label': 'LABEL_0', 'score': 0.962406575679779}]",LABEL_0,0.962407
260450,L3pSVjM-7LESrqSUu3E-3A,5.0,Lauren our waitress killed it. Served us up the best charbroiled oysters we've had on this side of the Mississippi and lobster that leaves you dreaming of lobster at night.,lauren our waitress killed it. served us up the best charbroiled oysters we've had on this side of the mississippi and lobster that leaves you dreaming of lobster at night.,37,"[{'label': 'LABEL_2', 'score': 0.9958130717277527}]",LABEL_2,0.995813
460473,NpVollkLK24mmyMtQlH69Q,3.0,"There is so much to like about this place. The atmosphere is amazing and the service has been consistently great and friendly. And the food looks beautiful. But there are simply two things I cannot get beyond, and unfortunately, I think they are killers for a restaurant.\n\nThe food is slightly above average at best, which is perfectly fine.\n\nBut the prices reflect food that should be better than it is. So, I would like to get that out of the way right now. \n\nI do not think the food is bad. \n\nI just don't think it's a great value. \n\nWith that out of the way, I have consumed the following at this place over my repeated attempts at ""giving 

In [17]:
# Sentiment Label Breakdown
test['label'].value_counts() # Mostly positive reviews

label
LABEL_2    3503
LABEL_0    1005
LABEL_1     492
Name: count, dtype: int64

In [18]:
# LABEL_0 only

query = """
select clean_text
, review_stars
, label
, score
from test
where label = 'LABEL_0'
"""

label_0 = duckdb.query(query).df()
label_0.head()

,clean_text,review_stars,label,score
0,"this was my second time at sabrina's during the covid crisis. both times i waited, but this time i was in line for over 2 1/2 hours. when i was finally seated, it took the waitress very long for me and my guest to even get water. i ordered the santa fe turkey burger. it was so extremely salty, that i was not able to eat it. i joked with my friend about not only the saltiness of the meal, but of the 3 pieces of black beans that were placed on the patty. when i asked to have it replaced, the turkey burger came with no beans, and a few pieces of corn. this was an overpriced turkey burger that was definitely not worth the wait.",2.0,LABEL_0,0.962407
1,awful got a taco salad had refried beans in it topped with tasteless ground beef sour cream guacomole and shredded cheese with shredded lettuce by the time i was a quarter through eating i needed a spoon if i wanted to finish because now it was a soup a gray nasty soup,1.0,LABEL_0,0.989698
2,"i have never reviewed a restaurant before but i felt this was deserved. extremely disappointing. went about 4:30 on a thursday. the place wasn't busy at all. waitress was polite but distracted. i ordered the pot roast hash and potato cheddar soup, unsweetened iced tea. my son ordered the border scramble.the waitress brought out the food. my son's looked like they poured cheap jarred salsa on it, all runny and swimming in liquid. no cheese, sour cream, or anything that was supposed to come with it. that includes the hash browns, which we had to remind the waitress to get. he was finished with the omelet by the time the hash browns showed up. they were obviously reheated in a microwave, barely cooked and cold. and no bread was offered even though it was supposed to be part of the meal.my hash came out as ordered, more or less. again, no bread as per the menu. she forgot about the soup until, after being reminded, she brought out what i can only guess was potatoes in some type of mac and cheese sauce. no bacon on top, no garnish of any type. brought out a sweetened iced tea instead of the unsweetened i had ordered. and that was after a second reminder. after dropping off the food she disappeared until it was time to get the check. never came to check on us, see how the food was, refills on drinks, nothing.there was a kid of about 7 or so running rampant around the restaurant. in the aisles, behind the counter, pretty much anywhere he wanted. i can only guess that he was a child of an employee. now i understand difficulties with child care but seriously? this is dangerous. went to pay and was asked ""how was the meal?"" by a different server. i said the complaints were too numerous so she asked if i wanted to talk to the mgr. i chose not to bother because after reading other reviews of this location it seemed a waste of time. i do regret that decision. i will be forwarding a copy of this to the corporate headquarters.",2.0,LABEL_0,0.988701
3,"food was not prepared right, i ordered 2 mcchickens with only cheese and bacon. when i left i realized there was lettuce added so i went to exchange it for the right food and the manager gave us attitude and snatched our debit card when we went to pay. definitely will not be back.",1.0,LABEL_0,0.975462
4,"the experience for first time going there on mother's day, was not good. we had to ask and then wait quite a long time to be served an offering of meat. and then after that the offering still came slowly for different kinds of meat. to get a drink or refill on coffee took an in ordinate amount of time. and then we waited for at least 25 minutes until we were able to settle our bill for someone to come bring our bill. i had to go ask to get the bill. the $250 bill i paid was not commensurate with the service received for the quality of food provided. i would not recommend anybody to go there.",1.0,LABEL_0,0.958227


In [19]:
label_0['review_stars'].value_counts() #for lowest star ratings/negative sentiment

review_stars
1.0    551
2.0    333
3.0    102
4.0     13
5.0      6
Name: count, dtype: int64

In [20]:
# Curious about possible mislabeling, what are the reviews acually reading as. was user accurate in their star rating or sentiment model wrong?
query = """
select clean_text
, review_stars
, label
, score
from label_0
where review_stars >=4
"""

bad_labels_0 = duckdb.query(query).df()
bad_labels_0

,clean_text,review_stars,label,score
0,"this place is dope af. beer. bratz, schnitzel, and some dank ass taters. you won't rugrat going to this place. screw dat dog. bratz y'all! ftw",5.0,LABEL_0,0.581289
1,harry the bartender. 8 months working there. second week at the bar. 1st food customer. messed up jumbalya instead of gumbo. corrected it before i began to eat the trio. good job!!,5.0,LABEL_0,0.967297
2,"stop offering me that turtle concrete shake thingy. please, just stop. i have no power to say no. :d management has really stepped it up here lately. faster, fresher. i just wish their corporate standards could include some garlic and cumin on their burgers!!!! solid burger joint. from a simpler time.",4.0,LABEL_0,0.455301
3,don't know why such a low review- cheesesteak was awesome the bread sealed the deal and put them ahead of pats for me.. lost a star due to signage- we notice the racist signs as soon as we got on the premises they definitely are off putting and not necessary the establishment should remove.,4.0,LABEL_0,0.811667
4,"came here on a late sunday with the guys - there was quite a line outside, maybe 10-15 people ahead of us but it was actually moving pretty quickly which was nice. when we finally got inside we saw that the place wasn't even really that full - maybe 70%. figures they were probably throttling the customers a little bit as to not overload the servers. anyway - we ended up getting seated upstairs which was cool. our drinks unfortunately took a long time to come out. our server said they ran out of root beer upstairs so.. i guess they had to go find some downstairs.. but it shouldn't take 15 min to walk a flight of stairs right? food came out shortly after the drinks and then the waitress disappeared for a while. she never came back to check on us by the time we finished our meal. we ended up having to flag 2 different people down just to find her. we did end up ordering some desserts and they came out ok. overall dining experience was ok, the place is pretty happenin' which is cool but the service could have been better. seems to be the trend in this neck of the woods with tip being added automatically to most places. hopefully you will get better service than we did.",4.0,LABEL_0,0.583049
5,"sandwiches are different and very good. but....customer service sucks! they seem unhappy, bothered, and annoyed with their job.",4.0,LABEL_0,0.733649
6,i went here a couple months ago. i had the pork chop. i'm not sure it's the same as what is on the menu now but i remember it had caramelized onions and smothered in bbq sauce. it tasted good but nothing to write home about. maybe that's my fault for ordering a bbq pork chop at a place called flatwater. i think my date ordered a salmon sandwich or something like that. she said it was very good.,4.0,LABEL_0,0.614192
7,"are they still open?? we went there three times (on different time slot) and they're always closed. agreed. in santa barbara area, this place has more better pho than others.",4.0,LABEL_0,0.454810
8,"i called the other day for pick up and they told me that prices had increased by 10%!!! during a pandemic, only thai son would increase their prices instead of doing something good for their customers. i've been coming here since they open and the food is delicious but this was a disappointment.",4.0,LABEL_0,0.802423
9,"complete 180 from the overpriced food and rude staff of molly maguires. stove and tap is exactly what a bar/restaurant should be. friendly service with affordable food and drink. kudos to everyone involved in creating this new ""must try"" in the heart of lansdale. i have been here several times and not once has the food been anything less than extraordinary. keep up the good work",5.0,LABEL_0,0.559798


# labeling check (sampled 6)
1. incorrect rating from model - 5 (0,1,3,4,5)
2. incorrect star rating from user - 1 (2)

In [21]:
bad_labels_0['score'].describe()

count    19.000000
mean      0.714765
std       0.186882
min       0.386118
25%       0.573045
50%       0.747079
75%       0.837136
max       0.992107
Name: score, dtype: float64

In [22]:
print('Percentage of bad labels in negative sentiment class:',round(len(bad_labels_0)/len(label_0),2) *100, '%.')

Percentage of bad labels in negative sentiment class: 2.0 %.


In [23]:
# LABEL_1 only

query = """
select clean_text
, review_stars
, label
, score
from test
where label = 'LABEL_1'
"""

label_1 = duckdb.query(query).df()
label_1.head()

,clean_text,review_stars,label,score
0,"there is so much to like about this place. the atmosphere is amazing and the service has been consistently great and friendly. and the food looks beautiful. but there are simply two things i cannot get beyond, and unfortunately, i think they are killers for a restaurant. the food is slightly above average at best, which is perfectly fine. but the prices reflect food that should be better than it is. so, i would like to get that out of the way right now. i do not think the food is bad. i just don't think it's a great value. with that out of the way, i have consumed the following at this place over my repeated attempts at ""giving it another chance."" mama's meatballs (small plate)ceci (small plate)margherita pizzablanco pizzapurple rain pizzanomad pizzapaulie gee pizzawedge saladcaesar salad as you can see, i am not making this opinion off of a one-off visit. i consistently gave the place a shot over and over because i like everything else about it so much. but ultimately, the taste never matched my wallet. i can strongly recommend eating anything they have on their menu that contains basil. they have wonderful basil.",3.0,LABEL_1,0.644316
1,"great service. nice people. the owner and his family were very friendly. i hate to say anything bad because i got a free boiling (and spicy!) dumpling bowl by checking-in on yelp. but, the truth is the truth, so here goes ... won ton soup was acceptable. nothing special.egg roll was fine ... and nicely (temperature) hot. but just a standard egg roll.the boiling dumplings were good ... and i thought that the sauce was very spicy (nice!). but i made the mistake of ordering the house special chung king chicken (not sure of the spelling). basically, it's just a mound of fried, diced chicken. tasty but very salty. and no vegetables. just fried chicken with a whole lot of dried peppers. i should've gotten the kung pao chicken like another yelp reviewer recommended.",3.0,LABEL_1,0.868553
2,this is a good surprise to find in a strip mall. more high end prices with good food. not sure it was worth the price but if you want fresh food they have it. lots of tvs gives it a pub feel which it out of place for an asian restaurant. they were not busy when we were there and not sure what that says for the place.,4.0,LABEL_1,0.813656
3,patio was nice but no actual view other than the road. service was friendly though not especially fast. the food was most disappointing for the price point. blood orange margarita was decent. clam chowder was good but very potato heavy. ceviche wasn't very flavorful- i asked for lime and salt to doctor it up myself. lobster mac and macadamia nut encrusted sea bass were both just mediocre at best. we also had the lava cake which was exceptionally rich but good. overall we just had higher hopes for the culinary experience and this restaurant overcharged and undelivered.,3.0,LABEL_1,0.614256
4,"went on a saturday night and was not surprised that it was a two hour wait even though dining room was half full. it's definitely the kind of place that takes good care of its many regulars. so even though it was a cool night i decided to eat outside with the other losers who couldn't get in. the food was good,it's always good just like the service. the problem is midway through my meal two women came out to smoke. the smoke coming directly over to my table. then two doofuses came out to smoke. cigars! at that point everyone on the patio was done eating and went inside to pay. c'mon gratzzi, if you're going to seat the patio and charge the same money as inside, show your diners enough respect to keep smokers, (cigar smokers!) away from where people are trying to eat. but i know you won't do anything. i bet they are regulars.",3.0,LABEL_1,0.717141


In [24]:
label_1['review_stars'].value_counts() #suppose to be for neutral. makes sense that 1 and 5 are on the bottom

review_stars
3.0    284
4.0    107
2.0     73
1.0     15
5.0     13
Name: count, dtype: int64

In [25]:
# Curious about possible mislabeling, what are the reviews acually reading as. was user accurate in their star rating or sentiment model wrong?
query = """
select clean_text
, review_stars
, label
, score
from label_1
where review_stars in (1,5)
"""

bad_labels_1 = duckdb.query(query).df()
bad_labels_1

,clean_text,review_stars,label,score
0,ordered an açaí bowl and all i got was a bowl of granola and fruit. maybe having more than 3 workers in the morning during busy months (summer) would create happier customers and better accuracy with orders. i mean how do you make an açaí bowl and not notice the bright purple açaí missing when you put the granola in smh.,1.0,LABEL_1,0.546689
1,"q: how many hipsters does it take to screw in a light bulb? a: it's an obscure number, you've probably never heard of it. i should say right away that the food at johnny brenda's was great. i've got no problem with the burger and fries i had there this afternoon. i do have a problem with lousy service. and today that's exactly what i got. i was meeting friends for lunch. the three of us sat outside, and i'm not quite sure what we did to anger the young girl that eventually waited on us, but i imagine it had something to do with the three of us not being ac neuman or win butler or kurt vile. we were clearly the least cool people in the place. although, it being lunchtime and a tuesday, for a while we were the only people there. i hope whoever you are, you're reading this, in the office before your shift, or on your iphone, or in the bathroom in between lines. you, with the ponytail and the ironic redskins shirt. i'm a server. i would never berate a customer for not knowing the menu, or ignore them for long periods of time, or clear half the table at a time, or basically anything you did when you waited on us. please understand the first rule of any service industry job: people forget bad food, but they never forget bad service.",1.0,LABEL_1,0.469273
2,"i always call in and ask for the fit bowl and get it without any problem, but for some reason today they would not let me call in and order this bowl saying it is only available through uber eats only. i don't understand what makes today so different, and it's not like it is so hard to just ring up the same things that are in the fit bowl for a ""custom bowl"". the service here is always hit or miss.",1.0,LABEL_1,0.639164
3,i used to loved getting jimmy johns at my other location. this place comes an hour after ordering and always misses something that i have ordered. m,1.0,LABEL_1,0.455289
4,"on the weekends they state that they have menudo and other soups available. it's on weekends only. went in today (sunday) evening and was told they have none. if it's weekends only then it should be offered all weekend, not just parts of it or it could say until it runs out or if available. usually always good food, but this is poor business as far as the menu is concerned and stated.",1.0,LABEL_1,0.551674
5,"the only reason i am giving this restaurant one star is because yelp requires it. minus the one star and it gets a zero. we were recommended to go to irene's by a very prominent nola citizen who absolutely raved and is a regular customer. several websites said that they did not take reservations and jackets were not required for men. we arrived promptly when they opened at 5:30 and not only did they tell us that they did in fact take reservations, and it would be 1 1/2 hours till they could seat us (the restaurant was empty) and there was no bar where we could go until our table was ready. the maître d was a bit snooty and gave me a once over like he didn't approve of the way i was dressed (casual with a skirt, heels and pretty blouse). i'm sure if i had mentioned who had recommended this restaurant, it would have been a whole nother story, but that's not how i roll. we left and went to cafe amelie where we had an amazing french meal, a wonderful bottle of wine and to-die-for desserts. take that, irene.",1.0,LABEL_1,0.504174
6,"very friendly , went for a mid morning breakfast, not crowded , food took longer then expected for simple order , also food was cold .",1.0,LABEL_1,0.779867
7,"the outside is fascinating, the inside is old and dirt. we left in a hurry after only one night. i let the pictures speak fo

# labeling check (sampled 6) 
1. incorrect rating from model - 6 (0,1,2,3,4,5)
2. incorrect star rating from user - 

In [26]:
bad_labels_1['score'].describe()

count    28.000000
mean      0.632205
std       0.119119
min       0.455289
25%       0.550428
50%       0.608062
75%       0.698181
max       0.874922
Name: score, dtype: float64

In [27]:
print('Percentage of bad labels in neutral sentiment class:',round(len(bad_labels_1)/len(label_1),2) *100, '%.')

Percentage of bad labels in neutral sentiment class: 6.0 %.


In [28]:
# LABEL_2 only

query = """
select clean_text
, review_stars
, label
, score
from test
where label = 'LABEL_2'
"""

label_2 = duckdb.query(query).df()
label_2.head()

,clean_text,review_stars,label,score
0,our waiter winston was kind and attentive to our visit. wow for winston!! five stars.,5.0,LABEL_2,0.997597
1,"who hasn't hung out at dargan's, i mean really! this is a great place to get in touch with your irish roots, though for me those are distant roots but just a hop and a skip on a ferry from england where i couldn't get a better serving of corn beef and cabbage if i searched out the finest of pups and all the land, dargan's is irie my friends. on st. patrick's day, when in santa barbara, it is a tradition to get you some shots of jamison's and green beer at the james joyce o'malley's and/or dargan's!!",5.0,LABEL_2,0.996617
2,lauren our waitress killed it. served us up the best charbroiled oysters we've had on this side of the mississippi and lobster that leaves you dreaming of lobster at night.,5.0,LABEL_2,0.995813
3,"i was hungry and got the gyro combo with the drink and fries and was not disappointed. it was very filling! next time i would just get the gyro because i was full from just the gyro itself. i forced myself (i know, big no no) too eat the fries because i'm not one to throw away food. it was also interesting to people watch. just don't stare too long lest you want some unnecessary roughness. the only thing missing was more sauce and veggies on my gyro. that's why a star is missing.",4.0,LABEL_2,0.958989
4,best restaurant in south shore hands down! food quality is great and you can bring your own wine. the decor and seats could be a little nicer but the food is the star here. great quality old-style italian food,5.0,LABEL_2,0.997959


In [29]:
label_2['review_stars'].value_counts() # suppose to be for highly positive

review_stars
5.0    2256
4.0    1077
3.0     141
2.0      15
1.0      14
Name: count, dtype: int64

In [30]:
# Curious about possible mislabeling, what are the reviews acually reading as. was user accurate in their star rating or sentiment model wrong?
query = """
select clean_text
, review_stars
, label
, score
from label_2
where review_stars in (1,2)
"""

bad_labels_2 = duckdb.query(query).df()
bad_labels_2

,clean_text,review_stars,label,score
0,if you want a sassy bartender who determines if your worthy of drinking one of his finest ocean watered down cocktails then you'll actually love this place. 12 year olds running around acting like they know how to run a restaurant. good luck..no wonder you're a widow.,1.0,LABEL_2,0.976755
1,"in-room dining was the most un-believable experience i'd ever had in 50+ years of business and pleasure travel. after 1-1/2 hour wait, the meal was brought to the room in a brown paper bag! the food was in plastic fast food take-out containers with plastic utensils. only comment from attendant was ""we've been doing this for a month or so"". a call to the room service operator resulted in being directed to an apologetic supervisor, who suggested i leave a complaint with the desk when i check out. i will say the newly re-modeled rooms were very nice, food in rosie's was pretty good and the engineering department really helped me with an exhibit show part that needed repairing. 1 star for ""pitiful"" in-room dining, 4 star for the rest of the stay.",1.0,LABEL_2,0.609334
2,"i eat here all the time. well, i used to. just parked in the back and the guy that's usually nice to me was yelling at me saying he was going to tow my truck! wth??. you can take your chile relleno and stuff it another way! small town with tons of mexican restaurants. adios, jerks!",1.0,LABEL_2,0.740330
3,"our neighborhood l.a. cafe has since found another new owner, and to be fair, we haven't eaten there in about the last 10-11 months. we tried its newest reincarnation a couple times, but the food didn't do much for us, although the portions were huge. perhaps someone else had a better experience and maybe it's improved since, but it's been hard to persuade ourselves to go back. old l.a. cafe, we miss you!",1.0,LABEL_2,0.788213
4,"went through the drive thru for an egg mcmuffin. lady tried to hand me an oj, and i told her all i had was an egg mcmuffin. she handed me a bag, and what do you know... i get to work and it's not an egg mcmuffin. geniuses! :(",1.0,LABEL_2,0.580194
5,"1/30/2019 update: i have found a solution to the 3 year debacle! toast on laurel street apparently makes the tartine king cakes the original way! a coworker brought one in today, and it is perfect! exactly as the original tartine used to make them. so get your tartine kingcakes from toast and enjoy their amazing ness!",2.0,LABEL_2,0.997896
6,stopped in here just to check the place out as i had never been in before. it smells wonderful. actually today walking in the door there was a definite bacon smell! we took a look at what was on offer and left as soon as we found out there were no donuts.,2.0,LABEL_2,0.891133
7,"i give dock street two stars, although we at least pick up a pizza here every month. why do we keep coming back? sheer desperation. dock street is the best pizza in the area. hands down. it's not cheap, it's not unique, we always get the same thing (the portabella and gorgonzola number), and the atmosphere is crummy. but there's really no place else to get a decent pizza around here. if you do need a good pizza and are in west philly, though, we'd recommend getting take-out. the space is big and looks as if it was originally intended to be industrial, but in a futile attempt to warm it up someone had amateur craft night and spattered their work all over the walls. the staff is, generally speaking, slow, and working too hard to affect how icily chic they are to serve you with any sensitivity or consideration. the beer is actually all right, but five bucks for beer brewed on the premises (which costs very little, despite the spiking cost of hops) and served in a tiny glass is just too much to bear. we hope never to have to eat in the restaurant itself again, although we'll certainly spend our hard-earned money on a palatable pizza, once in a while. that's the price one pays to live in west philly, i guess.",2.0,LABEL_2,0.514215
8,"i really like this n

# labeling check (sampled 6) 
1. incorrect rating from model - 6 (0,1,2,3,4,5)
2. incorrect star rating from user - 

In [31]:
bad_labels_2['score'].describe() # lower percentage the better since these not good labels from model

count    29.000000
mean      0.741752
std       0.179630
min       0.378904
25%       0.609334
50%       0.740330
75%       0.905763
max       0.997896
Name: score, dtype: float64

In [32]:
print('Percentage of bad labels in positive sentiment class:',round(len(bad_labels_2)/len(label_2),2) *100, '%.')

Percentage of bad labels in positive sentiment class: 1.0 %.


Model was still confident in some really bad/really good sentiment reviews,but only <5% for each sentiment class.

In [33]:
list(test)

['review_id',
 'review_stars',
 'text',
 'clean_text',
 'num_tokens_tok',
 'sentiment',
 'label',
 'score']

## Will continue to work with 'test" dataset (5,000 samples) due to limited CPU

In [34]:
# Changing sentiment labels to ones that make more sense
query = """
select review_id
, review_stars
, clean_text
, case when label = 'LABEL_0' then 'negative'
when label = 'LABEL_1' then 'neutral'
else 'positive'
end as hf_sent_label
, score as hf_sent_score
from test

"""

test2 = duckdb.query(query).df()
test2

,review_id,review_stars,clean_text,hf_sent_label,hf_sent_score
0,bZxahkC542f7D6xlDSLsnA,5.0,our waiter winston was kind and attentive to our visit. wow for winston!! five stars.,positive,0.997597
1,p1-uV5Iv97kUI5zqIJSy5A,5.0,"who hasn't hung out at dargan's, i mean really! this is a great place to get in touch with your irish roots, though for me those are distant roots but just a hop and a skip on a ferry from england where i couldn't get a better serving of corn beef and cabbage if i searched out the finest of pups and all the land, dargan's is irie my friends. on st. patrick's day, when in santa barbara, it is a tradition to get you some shots of jamison's and green beer at the james joyce o'malley's and/or dargan's!!",positive,0.996617
2,tdGwUnyIg6Y1gXKF1iMRkg,2.0,"this was my second time at sabrina's during the covid crisis. both times i waited, but this time i was in line for over 2 1/2 hours. when i was finally seated, it took the waitress very long for me and my guest to even get water. i ordered the santa fe turkey burger. it was so extremely salty, that i was not able to eat it. i joked with my friend about not only the saltiness of the meal, but of the 3 pieces of black beans that were placed on the patty. when i asked to have it replaced, the turkey burger came with no beans, and a few pieces of corn. this was an overpriced turkey burger that was definitely not worth the wait.",negative,0.962407
3,L3pSVjM-7LESrqSUu3E-3A,5.0,lauren our waitress killed it. served us up the best charbroiled oysters we've had on this side of the mississippi and lobster that leaves you dreaming of lobster at night.,positive,0.995813
4,NpVollkLK24mmyMtQlH69Q,3.0,"there is so much to like about this place. the atmosphere is amazing and the service has been consistently great and friendly. and the food looks beautiful. but there are simply two things i cannot get beyond, and unfortunately, i think they are killers for a restaurant. the food is slightly above average at best, which is perfectly fine. but the prices reflect food that should be better than it is. so, i would like to get that out of the way right now. i do not think the food is bad. i just don't think it's a great value. with that out of the way, i have consumed the following at this place over my repeated attempts at ""giving it another chance."" mama's meatballs (small plate)ceci (small plate)margherita pizzablanco pizzapurple rain pizzanomad pizzapaulie gee pizzawedge saladcaesar salad as you can see, i am not making this opinion off of a one-off visit. i consistently gave the place a shot over and over because i like everything else about it so much. but ultimately, the taste never matched my wallet. i can strongly recommend eating anything they have on their menu that contains basil. they have wonderful basil.",neutral,0.644316
...,...,...,...,...,...
4995,EZwEl2rTyxMRnDNSdcU7qA,4.0,"ok so i have been going down for pizza before the footballs games for years. i recently have been coming to no anchovies more and more and it's really growing on me. the only reason it has to grow on me was that i saw this place being built when i was in school and it really sucked before. harsh words but i went a few time back in 97-98 and this place was average at best. i spent most of my time next door to my still favorite frog and saying why can't no anchovies be something like them. fast forward it 15 years and no no anchovies is the place to be and i can see why! now it's right on par with frog and i'm alright with that! first off the outdoor patio is rocking with music, tvs, and they have a full bar with amazing craft beers. very good prices on them too. back in the 90s (wow i sound old!) and there was nothing like that back then. but the pizza was always just average in my memories but now i. really starting to like no anchovies pizza too. anyways i will come back here and it makes me happy that university drive now has two great places to relax or watch a game and eat great pizza and

In [35]:
# Let's see if Vader, popular sentiment score for social texts (tweets with emojoi's, slang, contractions)
analyzer = SentimentIntensityAnalyzer()
test2['vader_rating'] = test2['clean_text'].apply(analyzer.polarity_scores)

# Docs on how scoring was derived and what it means: https://github.com/cjhutto/vaderSentiment#about-the-scoring
# positive sentiment: compound score >= 0.05
# neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
# negative sentiment: compound score <= -0.05
test2['compound'] = [analyzer.polarity_scores(x)['compound'] for x in test2['clean_text']]
test2['neg'] = [analyzer.polarity_scores(x)['neg'] for x in test2['clean_text']]
test2['neu'] = [analyzer.polarity_scores(x)['neu'] for x in test2['clean_text']]
test2['pos'] = [analyzer.polarity_scores(x)['pos'] for x in test2['clean_text']]

In [36]:
test2.head()

,review_id,review_stars,clean_text,hf_sent_label,hf_sent_score,vader_rating,compound,neg,neu,pos
0,bZxahkC542f7D6xlDSLsnA,5.0,our waiter winston was kind and attentive to our visit. wow for winston!! five stars.,positive,0.997597,"{'neg': 0.0, 'neu': 0.625, 'pos': 0.375, 'compound': 0.8309}",0.8309,0.000,0.625,0.375
1,p1-uV5Iv97kUI5zqIJSy5A,5.0,"who hasn't hung out at dargan's, i mean really! this is a great place to get in touch with your irish roots, though for me those are distant roots but just a hop and a skip on a ferry from england where i couldn't get a better serving of corn beef and cabbage if i searched out the finest of pups and all the land, dargan's is irie my friends. on st. patrick's day, when in santa barbara, it is a tradition to get you some shots of jamison's and green beer at the james joyce o'malley's and/or dargan's!!",positive,0.996617,"{'neg': 0.032, 'neu': 0.889, 'pos': 0.079, 'compound': 0.6807}",0.6807,0.032,0.889,0.079
2,tdGwUnyIg6Y1gXKF1iMRkg,2.0,"this was my second time at sabrina's during the covid crisis. both times i waited, but this time i was in line for over 2 1/2 hours. when i was finally seated, it took the waitress very long for me and my guest to even get water. i ordered the santa fe turkey burger. it was so extremely salty, that i was not able to eat it. i joked with my friend about not only the saltiness of the meal, but of the 3 pieces of black beans that were placed on the patty. when i asked to have it replaced, the turkey burger came with no beans, and a few pieces of corn. this was an overpriced turkey burger that was definitely not worth the wait.",negative,0.962407,"{'neg': 0.057, 'neu': 0.859, 'pos': 0.084, 'compound': 0.6653}",0.6653,0.057,0.859,0.084
3,L3pSVjM-7LESrqSUu3E-3A,5.0,lauren our waitress killed it. served us up the best charbroiled oysters we've had on this side of the mississippi and lobster that leaves you dreaming of lobster at night.,positive,0.995813,"{'neg': 0.123, 'neu': 0.763, 'pos': 0.114, 'compound': -0.0772}",-0.0772,0.123,0.763,0.114
4,NpVollkLK24mmyMtQlH69Q,3.0,"there is so much to like about this place. the atmosphere is amazing and the service has been consistently great and friendly. and the food looks beautiful. but there are simply two things i cannot get beyond, and unfortunately, i think they are killers for a restaurant. the food is slightly above average at best, which is perfectly fine. but the prices reflect food that should be better than it is. so, i would like to get that out of the way right now. i do not think the food is bad. i just don't think it's a great value. with that out of the way, i have consumed the following at this place over my repeated attempts at ""giving it another chance."" mama's meatballs (small plate)ceci (small plate)margherita pizzablanco pizzapurple rain pizzanomad pizzapaulie gee pizzawedge saladcaesar salad as you can see, i am not making this opinion off of a one-off visit. i consistently gave the place a shot over and over because i like everything else about it so much. but ultimately, the taste never matched my wallet. i can strongly recommend eating anything they have on their menu that contains basil. they have wonderful basil.",neutral,0.644316,"{'neg': 0.059, 'neu': 0.7, 'pos': 0.242, 'compound': 0.9911}",0.9911,0.059,0.700,0.242
